In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import torch.nn.functional as F

In [2]:
# Fix Random Seeds
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    device = torch.device("cuda")
    print("CUDA is available")
else:
    torch.device("cput")
    print("NO CUDA is available")   

CUDA is available


In [3]:
# model_name = r"C:\LLMs\Meta-Llama-3-8B-Instruct"
# model_path = "./pretrained_llms/Llama-3.3-70B-Instruct"
model_path = "./pretrained_llms/Llama-3.1-8B-Instruct"
data_path = "./data"
data_name = "mteb/tweet_sentiment_extraction"
cache_dir = "./cache"


In [4]:
dataset = load_dataset(data_name, cache_dir=data_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, torch_dtype="auto").to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          add_eos_token=True,
                                          cache_dir=cache_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ./pretrained_llms/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# print(dataset)

In [5]:
if tokenizer.pad_token_id is None:
    print("No pad token found, setting pad token to eos token")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "right"
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

def tokenizer_function(examples):
    return tokenizer(examples['text'],
                     return_tensors="pt",
                     truncation=True,
                     padding=True)


# apply tokenizer function on your data
tokenized_data = dataset.map(tokenizer_function, batched=True, remove_columns=["id"])
train = tokenized_data['train']
test = tokenized_data['test']

No pad token found, setting pad token to eos token


Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [6]:
train[7]

{'text': 'Soooo high',
 'label': 1,
 'label_text': 'neutral',
 'input_ids': [128000,
  50,
  26845,
  1579,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009],
 'attention_mask': [1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [7]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
print(terminators)
print(
    f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}"
)
print(
    f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}"
)

[128009, 128009]
Pad Token id: 128009 and Pad Token: <|eot_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


In [8]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [7]:
# inputs = tokenizer(train[6]['text'], return_tensors="pt", truncation=True, padding=True)
# Perform inference

example = train[1]
input_ids = torch.tensor([example['input_ids']]).to(device)
attention_mask = torch.tensor([example['attention_mask']]).to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Get predicted label
logits = outputs.logits
probabilities = F.softmax(logits, dim=-1)
predicted_label = torch.argmax(probabilities, dim=-1).item()
predicted_label_text = model.config.id2label[predicted_label]
print(predicted_label)

0


In [22]:
print(predicted_label_text)

LABEL_1


In [3]:
from transformers import DataCollatorWithPadding, GPT2Config, GPT2ForSequenceClassification
## When training a transformer model,
# it’s common to batch sequences together for more efficient processing.
# However, since sequences might have different lengths, they need to be padded to a common length within each batch.
#The DataCollatorWithPadding class automates this process.

#define the collator, use DataCollatorWithPadding() with the defined tokenizer above
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
config = GPT2Config()

#define GPT classifier, use 'gpt2' pretrained LLM, we have 3 classes in our dataset

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)
model.config.pad_token_id = model.config.eos_token_id

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import evaluate
import numpy as np

# load the accuracy metric
metric = evaluate.load("accuracy")


def acc(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
from transformers import TrainingArguments, Trainer

# training arguments: https://huggingface.co/docs/transformers/v4.40.2/en/main_classes/trainer#transformers.TrainingArguments
# set per_device_train_batch_size  per_device_eval_batch_size as 8,
# we will fine tune gpt model for 30 epochs , set the corresponding parameter
training_args = TrainingArguments(
    output_dir="test_trainer",
    #evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    save_total_limit=2
    #gradient_accumulation_steps=4
)

# set the data_collator and training arguments we defined above
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    compute_metrics=acc,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
# if it outputs memory error: shut down all kernels or restart kernel
# train the model, use train() function
trainer.train()

Step,Training Loss
500,0.626900
1000,0.104400
1500,0.008100
2000,0.002700
2500,0.001500
3000,0.001600
3500,0.001200


TrainOutput(global_step=3750, training_loss=0.09952616239984831, metrics={'train_runtime': 419.4324, 'train_samples_per_second': 71.525, 'train_steps_per_second': 8.941, 'total_flos': 513256776597504.0, 'train_loss': 0.09952616239984831, 'epoch': 30.0})

In [7]:
import evaluate

#evaluate the fine tuned model on test data; use evaluate
trainer.evaluate()

{'eval_loss': 3.218705177307129,
 'eval_accuracy': 0.703,
 'eval_runtime': 3.8726,
 'eval_samples_per_second': 258.225,
 'eval_steps_per_second': 32.278,
 'epoch': 30.0}

In [8]:
from transformers import TextClassificationPipeline

#test with new data, use TextClassificationPipeline to prepare your input text

# This pipeline has a return_all_scores parameter on its __call__ method that allows you to get all scores for each label on a prediction.
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# pipe your text here
prediction = pipe(
    "I find great pleasure in learning through the NLP courses offered by HLRS.",
    return_all_scores=True)
prediction

/opt/conda/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 2.2688107392809798e-08},
  {'label': 'LABEL_1', 'score': 6.458856660174206e-05},
  {'label': 'LABEL_2', 'score': 0.9999353885650635}]]